In [3]:
import numpy as np
from osgeo import ogr, gdal
import rasterio
import json

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os.path
import re

from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr

In [5]:
area_cover = '/home/shrayank_mistry/Modules/Area-routes/ASalem/mask.tif'
area = rasterio.open(area_cover, count = 1)

area = np.array(area.read(1))

In [6]:
area_dem = '/home/shrayank_mistry/Modules/Area-routes/ASalem/dem_clipped.tif'
dem = rasterio.open(area_dem, count = 1)

dem = np.array(dem.read(1))
print(dem.shape)

(5337, 5036)


In [7]:
import rasterio
from rasterio.enums import Resampling

# whole-numbers indicate upscaling, fractions indicate downscaling
upscale_factor = 3


with rasterio.open('/home/shrayank_mistry/Modules/Area-routes/ASalem/dem_clipped.tif') as dataset:

    # resample data to target shape
    data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * upscale_factor),
            int(dataset.width * upscale_factor)
        ),
        resampling=Resampling.bilinear
    )

    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )

In [8]:
area = np.swapaxes(area, 1, 0)
print(area.shape)

up_dem = data[0]
up_dem.shape

(15043, 16233)


(16011, 15108)

In [9]:
dem_data, mask_data = up_dem, area
dem_data = dem_data.T


print(dem_data.shape, mask_data.shape)
width, height = 15043, 16011

dem_data, mask_data = dem_data[:width,:height], mask_data[:width, :height]
print(dem_data.shape, mask_data.shape)

(15108, 16011) (15043, 16233)
(15043, 16011) (15043, 16011)


In [10]:
ahp_map = {

'urban': 0.29,
'farms': 0.239,
'dense-forest': 0.207,
'water': 0.13,
'fallow': 0.067,
'sparse-forest': 0.049,
'barren-land': 0.019,
'unclassified':7,
'unclassified2': 8,

}

class_map = {

    -999: 'unclassified2',
    0: 'unclassified',
    1: 'water',
    2: 'dense-forest',
    3: 'sparse-forest',
    4: 'barren-land',
    5: 'urban',
    6: 'farms',
    7: 'fallow',

}


In [11]:
from copy import deepcopy
mask_copy = deepcopy(mask_data)

In [12]:
def set_weights(c):
    c_str = class_map.get(c)
    return ahp_map[c_str]

In [13]:
set_weights_vtr = np.vectorize(set_weights)
mask_copy = set_weights_vtr(mask_copy)

In [14]:
np.unique(mask_copy, return_counts = True)

(array([0.019, 0.049, 0.067, 0.13 , 0.207, 0.239, 0.29 , 7.   , 8.   ]),
 array([75677893,  7449758, 44631533,  7691351, 45743885, 47330872,
        12328163,        6,       12]))

In [15]:
np.unique(mask_data, return_counts = True)

(array([-999,    0,    1,    2,    3,    4,    5,    6,    7], dtype=int16),
 array([      12,        6,  7691351, 45743885,  7449758, 75677893,
        12328163, 47330872, 44631533]))

In [16]:
from osgeo import ogr
import json

file = ogr.Open("/home/shrayank_mistry/Modules/Area-routes/ASalem/source/source.shp")
source_shp = file.GetLayer(0)

feature = source_shp.GetFeature(0)
source_shp = feature.ExportToJson()

# source_shp
source_shp = json.loads(source_shp)
start_ext = source_shp['geometry']['coordinates']

In [17]:
file = ogr.Open("/home/shrayank_mistry/Modules/Area-routes/ASalem/destination/destination.shp")
destination_shp = file.GetLayer(0)

feature = destination_shp.GetFeature(0)
destination_shp = feature.ExportToJson()

destination_shp

# source_shp
destination_shp = json.loads(destination_shp)
end_ext = destination_shp['geometry']['coordinates']

In [18]:
print(start_ext, end_ext)

[892086.4271144224, 1402194.1872975053] [849204.4494360032, 1290500.4032451797]


In [19]:
from osgeo import ogr, gdal
import os
import subprocess
import rasterio

In [20]:
path = '/home/shrayank_mistry/Modules/Area-routes/ASalem/raster.tif' 

data = rasterio.open(path)
print(data.bounds)

extent = data.bounds

left, bottom, right, top = extent[0], extent[1], extent[2], extent[3]
print(left, bottom, right, top)

BoundingBox(left=807951.0194687467, bottom=1255157.4422908756, right=958381.0194687467, top=1417487.4422908756)
807951.0194687467 1255157.4422908756 958381.0194687467 1417487.4422908756


In [21]:
width = round(right - left)
height = round(top - bottom)

print("Width and Height of Raster")
print(width, height)

Width and Height of Raster
150430 162330


In [22]:
# width = round(right - left)
# height = round(top - bottom)

# print(width, height)

# start_pixel, end_pixel = [0, 0], [0, 0]
# start_flag, end_flag = True, True

# for i in range(height):
#     for j in range(width):

#         if (not start_flag) and (not end_flag):
#             break
        
#         # print(round(left + j), round(top - i))
#         if (start_flag and ((round(left + j)) == round(start_ext[0])) and ((round(top - i)) == round(start_ext[1]))):
#             start_pixel = [i, j]
#             start_flag = False

        
#         if (end_flag and ((round(left + j)) == round(end_ext[0])) and ((round(top - i)) == round(end_ext[1]))):
#             end_pixel = [i, j]
#             end_flag = False

In [23]:
# start_pixel, end_pixel = [15293, 84135], [126987, 41253]
start_pixel, end_pixel = [84135, 15293], [41253, 126987]

In [24]:
start_pixel = np.array(start_pixel)
start_pixel = start_pixel / 10

start_pixel = list(np.rint(start_pixel))
start_pixel = list(np.array(start_pixel, dtype = 'int'))

end_pixel = np.array(end_pixel)
end_pixel = end_pixel / 10

end_pixel = list(np.rint(end_pixel))
end_pixel = list(np.array(end_pixel, dtype = 'int'))

### Dijkstra's Algorithm for Anistropic Accumulated-Cost Calculation

In [25]:
def condition_check(start, end):
    if (start[0] == end[0]) and (start[1] == end[1]):
        return False
    return True

In [26]:
import math
from queue import PriorityQueue

In [27]:
from copy import deepcopy
mask_copy_t = deepcopy(mask_copy)

In [28]:
mask_copy = mask_copy * 5000

In [128]:
# mask_copy = mask_copy.T
# print(mask_copy.shape)

# dem_data = dem_data.T
# print(dem_data.shape)

In [29]:
np.unique(mask_copy, return_counts = True)

(array([   95.,   245.,   335.,   650.,  1035.,  1195.,  1450., 35000.,
        40000.]),
 array([75677893,  7449758, 44631533,  7691351, 45743885, 47330872,
        12328163,        6,       12]))

In [30]:
def c_anist_cost(i, j, x, y, mu = 10, wt = 2):

    mu_sqr = mu * mu
    h_diff = dem_data[i][j] - dem_data[x][y]
    h_sqr = h_diff * h_diff
    c_dv = (mask_copy[i][j] + mask_copy[x][y]) / 2
    cst = np.sqrt(mu_sqr + h_sqr) * (c_dv + math.atan(h_diff / mu) * wt) + acc_cost[i][j]

    # print(cst)
    return cst


In [31]:
def get_neigh_cost(i, j, lt_iw, lt_jh):
    arr = []
    #(1) col - 1, row - 1
    if (j - 1 >= 0) and (i - 1 >= 0):
        # arr.append([c_anist_cost(i, j, i - 1, j - 1), [i - 1, j - 1], [i, j]])
        # acc_cost[i - 1][j - 1] = min(c_anist_cost(i, j, i - 1, j - 1), acc_cost[i - 1][j - 1])

        if (acc_cost[i - 1][j - 1] > c_anist_cost(i, j, i - 1, j - 1)):
            parent[i - 1][j - 1] = i, j
            acc_cost[i - 1][j - 1] = c_anist_cost(i, j, i - 1, j - 1)
        arr.append([acc_cost[i - 1][j - 1], [i - 1, j - 1], [i, j]])
    else:
        arr.append(math.inf)

    #(2) col, row - 1
    if (i - 1 >= 0):
        # arr.append([c_anist_cost(i, j, i - 1, j), [i - 1, j], [i, j]])
        # acc_cost[i - 1][j] = min(c_anist_cost(i, j, i - 1, j), acc_cost[i - 1][j])

        if (acc_cost[i - 1][j] > c_anist_cost(i, j, i - 1, j)):
            parent[i - 1][j] = i, j
            acc_cost[i - 1][j] = c_anist_cost(i, j, i - 1, j)
        arr.append([acc_cost[i - 1][j], [i - 1, j], [i, j]])
    else:
        arr.append(math.inf)

    #(3) col + 1, row - 1
    if (j + 1 < lt_jh) and (i - 1 >= 0):
        # arr.append([c_anist_cost(i, j, i - 1, j + 1), [i - 1, j + 1], [i, j]])
        # acc_cost[i - 1][j + 1] = min(c_anist_cost(i, j, i - 1, j + 1), acc_cost[i - 1][j + 1])

        if (acc_cost[i - 1][j + 1] > c_anist_cost(i, j, i - 1, j + 1)):
            parent[i - 1][j + 1] = i, j
            acc_cost[i - 1][j + 1] = c_anist_cost(i, j, i - 1, j + 1)
        arr.append([acc_cost[i - 1][j + 1], [i - 1, j + 1], [i, j]])
    else:
        arr.append(math.inf)

    #(4) col - 1, row
    if (j - 1 >= 0):
        # arr.append([c_anist_cost(i, j, i, j - 1), [i, j - 1], [i, j]])
        # acc_cost[i][j - 1] = min(c_anist_cost(i, j, i, j - 1), acc_cost[i][j - 1])

        if (acc_cost[i][j - 1] > c_anist_cost(i, j, i, j - 1)):
            parent[i][j - 1] = i, j
            acc_cost[i][j - 1] = c_anist_cost(i, j, i, j - 1)
        arr.append([acc_cost[i][j - 1], [i, j - 1], [i, j]])
    else:
        arr.append(math.inf)

    #(5) col + 1, row
    if (j + 1 < lt_jh):
        # arr.append([c_anist_cost(i, j, i, j + 1), [i, j + 1], [i, j]])
        # acc_cost[i][j + 1] = min(c_anist_cost(i, j, i, j + 1), acc_cost[i][j + 1])

        if (acc_cost[i][j + 1] > c_anist_cost(i, j, i, j + 1)):
            parent[i][j + 1] = i, j
            acc_cost[i][j + 1] = c_anist_cost(i, j, i, j + 1)
        arr.append([acc_cost[i][j + 1], [i, j + 1], [i, j]])
    else:
        arr.append(math.inf)

    #(6) col - 1, row + 1
    if (j - 1 >= 0) and (i + 1 < lt_ih):
        # arr.append([c_anist_cost(i, j, i + 1, j - 1), [i + 1, j - 1], [i, j]])
        # acc_cost[i + 1][j - 1] = min(c_anist_cost(i, j, i + 1, j - 1), acc_cost[i + 1][j - 1])

        if (acc_cost[i + 1][j - 1] > c_anist_cost(i, j, i + 1, j - 1)):
            parent[i + 1][j - 1] = i, j
            acc_cost[i + 1][j - 1] = c_anist_cost(i, j, i + 1, j - 1)
        arr.append([acc_cost[i + 1][j - 1], [i + 1, j - 1], [i, j]])
    else:
        arr.append(math.inf)

    #(7) col, row + 1
    if (i + 1 < lt_ih):
        # arr.append([c_anist_cost(i, j, i + 1, j), [i + 1, j], [i, j]])
        # acc_cost[i + 1][j] = min(c_anist_cost(i, j, i + 1, j), acc_cost[i + 1][j])

        if (acc_cost[i + 1][j] > c_anist_cost(i, j, i + 1, j)):
            parent[i + 1][j] = i, j
            acc_cost[i + 1][j] = c_anist_cost(i, j, i + 1, j)
        arr.append([acc_cost[i + 1][j], [i + 1, j], [i, j]])
    else:
        arr.append(math.inf)

    #(8) col + 1, row + 1
    if (j + 1 < lt_jh) and (i + 1 < lt_ih):
        # arr.append([c_anist_cost(i, j, i + 1, j + 1), [i + 1, j + 1], [i, j]])
        # acc_cost[i + 1][j + 1] = min(c_anist_cost(i, j, i + 1, j + 1), acc_cost[i + 1][j + 1])

        if (acc_cost[i + 1][j + 1] > c_anist_cost(i, j, i + 1, j + 1)):
            parent[i + 1][j + 1] = i, j
            acc_cost[i + 1][j + 1] = c_anist_cost(i, j, i + 1, j + 1)
        arr.append([acc_cost[i + 1][j + 1], [i + 1, j + 1], [i, j]])
    else:
        arr.append(math.inf)
    

    return arr

In [32]:
# start_pixel, end_pixel = [1770, 1442], [8344, 5485]
print(start_pixel, end_pixel)

[8414, 1529] [4125, 12699]


In [33]:
dim = mask_copy.shape
print(dim)

(15043, 16011)


In [34]:
print(mask_copy.shape, dem_data.shape)

(15043, 16011) (15043, 16011)


In [35]:
visited = np.zeros((dim))
acc_cost = np.full((dim), math.inf)

print(visited.shape, acc_cost.shape)

(15043, 16011) (15043, 16011)


In [36]:
print(start_pixel, end_pixel)

[8414, 1529] [4125, 12699]


In [37]:
# start_pixel, end_pixel = [8414, 1529], [8646, 1611]
start_pixel, end_pixel = [8414, 1529], [4125, 12699]
print(start_pixel, end_pixel)

[8414, 1529] [4125, 12699]


In [38]:
print(width, height)

150430 162330


In [39]:
wd, ht = int(width / 10), int(height / 10)
print(wd, ht)

15043 16233


In [ ]:
last_pixel = None

In [40]:
Q = PriorityQueue()

# Width and Height Limit of area
lt_ih, lt_jh = wd, ht
s_pixel, e_pixel = start_pixel, end_pixel

# mask_copy.shape, dem_data.shape

visited = np.zeros((dim))
acc_cost = np.full((dim), math.inf)

acc_cost[s_pixel[0]][s_pixel[1]] = 0
visited[s_pixel[0]][s_pixel[1]] = 1

parent = np.full((dim), None)
parent[s_pixel[0]][s_pixel[1]] = -1, -1


path = list()

while (condition_check(s_pixel, e_pixel)):
    # print('in')
    i, j = s_pixel
    # print(i, j)

    neighbours_cost = get_neigh_cost(i, j, lt_ih, lt_jh)
    for nc in neighbours_cost:
        if nc == math.inf:
            continue
        else:
            Q.put(nc)
    
    # check if current-best is visited OR not
    bst = Q.get()
    m, n = bst[1][0], bst[1][1]

    while True:
        if visited[m][n] == 0:
            break
        bst = Q.get()
        m, n = bst[1][0], bst[1][1]

    # set-visited
    visited[m][n] = 1
    path.append([m, n])
    # print(m, n)

    parent[m][n] = bst[2][0], bst[2][1]

    s_pixel = [m, n]
    last_pixel = s_pixel

KeyboardInterrupt: 

In [42]:
end_pixel = last_pixel

In [43]:
print(acc_cost[end_pixel[0]][end_pixel[1]])

5429604.797018611


In [44]:
!rm -rf path.txt

In [45]:
path = []
pr = parent[end_pixel[0]][end_pixel[1]]

# cnt = 15000
while (pr[0] != -1) and (pr[1] != -1):
    path.append(pr)
    # path.append('-')
    with open('/home/shrayank_mistry/Modules/Area-routes/ASalem/routes-shape/dijk-route/path.txt', 'a') as f:
        f.write(str(pr) + '\n')

    # cnt = cnt - 1
    # if cnt == 0:
    #     break

    pr = parent[pr[0]][pr[1]]

In [46]:
path_list = []
with open('/home/shrayank_mistry/Modules/Area-routes/ASalem/routes-shape/dijk-route/path.txt', 'r') as f:
    for point in f:
        path_list.append(point)

In [47]:
len(path_list)

7346

In [48]:
point = path_list[0].replace('\n', '').split(' ')
pi, pj = int(point[0].split(',')[0].split('(')[1]), int(point[1].split(',')[0].split(')')[0])
print(pi, pj)

7443 6781


In [49]:
path_list.reverse()

In [50]:
%%time
ordinates_dict = {}


for i in range(len(path_list)):
    point = path_list[i].replace('\n', '').split(' ')
    pi, pj = int(point[0].split(',')[0].split('(')[1]), int(point[1].split(',')[0].split(')')[0])

    ext_i, ext_j = left + (pi * 10), top - (pj * 10)

    # ordinates_dict[(pi, pj)] = extent_matrix[pi][pj]
    ordinates_dict[pi, pj] = [ext_i, ext_j]

CPU times: user 18.4 ms, sys: 4 µs, total: 18.4 ms
Wall time: 18.2 ms


In [51]:
ordinates_dict
ordinates_list = []
for key, value in ordinates_dict.items():
    ordinates_list.append(value)

In [52]:
import shapefile
w = shapefile.Writer('/home/shrayank_mistry/Modules/Area-routes/ASalem/routes-shape/dijk-route/shapefiles/test/multipoint')
w.field('name', 'C')

w.multipoint(ordinates_list) 
w.record('multipoint1')

w.close()

In [53]:
road_lenght = 0
index = 1
for _ in ordinates_list[1:]:
    i, j = ordinates_list[index][0], ordinates_list[index][1]
    x, y = ordinates_list[index - 1][0], ordinates_list[index - 1][1]

    if (x - i == 10.0) and (y - j == -10.0):
        road_lenght += math.sqrt(2 * 100)
    else:
        road_lenght += 10 

In [54]:
print(f'Current Road-length = {road_lenght / 1000} kms')

Current Road-length = 103.87398615629492 kms
